# Week 10 Quiz

## Weichen Li - wl2726

### Due Sun. April 26, 11:59pm

In this quiz, we're going to load documents from 2 topics (space, cars) in the 20newsgroups dataset.

The goal is to train a classifier that classifies documents into these 2 topics based on a term frequency representation of the documents.

We will then calculate mean cross-validaion accuracy of a RandomForestClassifier using this transformation.

### Setup Environment

In [1]:
import numpy as np
import pandas as pd

### Load the Dataset

In [2]:
# Import fetch_20newsgroups from sklearn.datasets
from sklearn.datasets import fetch_20newsgroups

# Load the dataset into newsgroups using fetch_20newsgroups.
#   Only fetch the training subset using subset='train'.
#   Only fetch the two topics using categories=['sci.space','rec.autos']
# Store in the result into newsgroups
newsgroups=fetch_20newsgroups( subset='train', categories=['sci.space','rec.autos'])

# Store the newsgroups.data as docs, newsgroups.target as y and newsgroups.target_names as y_names
docs=newsgroups.data
y_names = newsgroups.target_names
y=newsgroups.target

# Print the number of observations by printing the length of docs
len(docs)

1187

In [3]:
# Print the text of the first document in docs.
docs[1]

"From: smith@ctron.com (Lawrence C Smith)\nSubject: Re: MR2 - noisy engine.\nOrganization: Cabletron Systems, Inc.\nLines: 16\nDistribution: world\nReply-To: smith@ctron.com\nNNTP-Posting-Host: glinda.ctron.com\n\nIn article <Apr21.053718.19765@engr.washington.edu>, eliot@lanmola.engr.washington.edu (eliot) writes:\n\n>if the noise really bugs you, there is nothing else that you can do\n>except to sell it and get a V6.\n\nPerhaps a nice used '88 Pontiac Fiero GT?  2.8 liters.\n\nDoes anyone know if the motor mounts for the 2.8 and the twin-dual-cam 3.4\nliter match?  The 3.4 is supposedly derived from the pushrod 3.1, which was\na punched out 2.8 liter.  Should be a drop-in replacement, eh?  205 horses in\na mid-engine the size of a Fiero?\n\nLarry Smith (smith@ctron.com)  No, I don't speak for Cabletron.  Need you ask?\n-\nLiberty is not the freedom to do whatever we want,\nit is the freedom to do whatever we are able.\n"

In [4]:
# Print the target value of the first document in y.
y[1]

0

In [5]:
# Print the target_name of the first document using y_names and y.
y_names[1]

'sci.space'

### Use CountVectorizer to Convert To TF

In [6]:
# Import CountVectorizer from sklearn.feature_extraction
from sklearn.feature_extraction.text import CountVectorizer

# Initialize a CountVectorizer object. It should
#   lowercase all text, 
#   include both unigrams and bigrams
#   exclude terms that occur in fewer than 10 documents
#   exclude terms that occur in more that 95% of documents
#   exclude all 'english' stopwords
# Store as cvect

cvect = CountVectorizer(lowercase=True,
                        ngram_range=(1, 2),
                        min_df=10/len(docs),
                        max_df=0.95,
                        stop_words='english')



# Fit cvect on docs and transform docs into their term frequency representation.
# Store as X_tf
X_tf = cvect.fit_transform(docs)

# Print the shape of X_tf. 
# The number of rows should match the number of documents 
#    and the number of columns should be in the thousands
X_tf.shape

(1187, 3628)

In [7]:
# The stopwords learned by cvect are stored as a set in cvect.stop_words_
# We'd like to print out a small subset of these terms.
# One way to get a subset of a set is to treat it as a list.
# First, convert the stop_words_ set to a list.
# Store as stop_words_list
stop_words_list=list(cvect.stop_words_)
# Print out the first 5 elements in stop_words_list.
# Note that, since a set is unordered, 
#    there is no meaning to the ordering of these terms and they may vary over runs.
stop_words_list[:5]

['cs virginia',
 '304 2989',
 'info mazda',
 'removing particulate',
 'rockwell structural']

### Calculate Mean CV Accuracy Using RandomForestClassifier

In [8]:
# Import cross_val_score from sklearn.model_selection
from sklearn.model_selection import cross_val_score

# Import RandomForestClassifier from sklearn.ensemble
from sklearn.ensemble import RandomForestClassifier

# Get a set of 5-fold CV scores using
#    a RandomForestClassifier with 50 trees, X_tf and y
# Store as cv_scores
rf=RandomForestClassifier(n_estimators=50)
cv_scores=cross_val_score(rf,X_tf,y,cv=5)

# Print the mean of these cv_scores. The mean accuracy should be above .9
cv_scores.mean()

0.9679997379771474

### Optional: Find Important Features

In [9]:
# CountVectorizer stores the feature names (terms in the vocabulary) in two ways:
#  1. as a dictionary of term:colum_index pairs, accessed via cvect.vocabulary_
#  2. as a list of terms, in column index order, accessed via cvect.get_feature_names()
#
# We can get the indices of the most important features by retraining a RandomForestClassifier on X,y
#  and accessing .feature_importances_
#
# Using some combination of the above data-structures, 
#  print out the top 10 terms in the vocabulary
#  ranked by the feature importances learned by a RandomForestClassifier
# 
# The terms you find will likely not be surprising given the document categories.

importances = rf.fit(X_tf,y).feature_importances_
df = pd.DataFrame(list(zip(cvect.vocabulary_, importances)), 
               columns =['feature name', 'feature importance']) 
df.nlargest(10, columns=['feature importance'])

,feature name,feature importance
2980,civil,0.077848
626,cmu edu,0.040093
2201,guest,0.036835
641,doug,0.021736
2341,purdue edu,0.018573
2827,tedebear,0.016540
2158,programs,0.015468
1101,facts,0.013521
1354,ride,0.011083
229,land,0.010789
